#### <font color="#03997A"> Diplomado en Ciencia de datos UNAM </font>

##### <font color="#03997A"> Dr: Alejandro Pimentel  </font>

##### <font color="#03997A"> Modulo 8 Introducción al Deep Learning </font>

##### <font color="#03997A"> Tema 4 CNN  Alumno: Ibarra Ramírez Sergio </font>

In [1]:
import tensorflow as tf
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import normalize
from sklearn.preprocessing import OneHotEncoder

X,y = fetch_openml('mnist_784', as_frame=False, return_X_y=True)

X = normalize(X).reshape(-1,28,28,1)

X_train = X[:60000]
X_test = X[60000:]
y_train = y[:60000]
y_test = y[60000:]

codificador = OneHotEncoder(handle_unknown='ignore',sparse=False)
y_train = codificador.fit_transform(y_train.reshape(-1,1))
y_test = codificador.transform(y_test.reshape(-1,1))

X_train = tf.constant(X_train, dtype=tf.float32)
X_test = tf.constant(X_test, dtype=tf.float32)
y_train = tf.constant(y_train, dtype=tf.float32)
y_test = tf.constant(y_test, dtype=tf.float32)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
batch_size = 100 # Hiperparámetro
train_dataset = train_dataset.shuffle(batch_size).batch(batch_size)

c:\Users\Sergio\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(
c:\Users\Sergio\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Definicion y entrenamiento de una CNN de una capa 

In [2]:
b = tf.Variable(tf.random.normal([10]))
W = tf.Variable(tf.random.normal([1568,10]))

filtros = tf.Variable(tf.random.normal([4,4,1,8])) # (alto,ancho,canales,num_filtros)
b_conv = tf.Variable(tf.random.normal([8]))

optimizador = tf.keras.optimizers.experimental.SGD(learning_rate=0.1)

epochs = 10

for _ in range(epochs):
  loss_epoch = 0
  for x,y in train_dataset:
    with tf.GradientTape() as gt:
      # Capa1
      salida_conv = tf.nn.conv2d(x,filtros,[1,2,2,1],"SAME")
      salida_conv += b_conv

      # Omití la activación no lineal


      # Capa Final
      #  Se aplana la salida de la capa anteior
      salidaPlana = tf.reshape(salida_conv,[-1,1568])
      operacion_matricial = tf.matmul(salidaPlana,W)+b

      x_ent = tf.nn.softmax_cross_entropy_with_logits(
          logits = operacion_matricial,
          labels = y
      )

      loss = tf.reduce_mean(x_ent)
      optimizador.minimize(loss,[W,b,filtros,b_conv],gt) # Poderosa

    loss_epoch += loss/len(train_dataset)
  print(loss_epoch)

tf.Tensor(8.785183, shape=(), dtype=float32)
tf.Tensor(0.65871847, shape=(), dtype=float32)
tf.Tensor(0.53738683, shape=(), dtype=float32)
tf.Tensor(0.48027182, shape=(), dtype=float32)
tf.Tensor(0.4459387, shape=(), dtype=float32)
tf.Tensor(0.4228714, shape=(), dtype=float32)
tf.Tensor(0.40497893, shape=(), dtype=float32)
tf.Tensor(0.39178798, shape=(), dtype=float32)
tf.Tensor(0.38067207, shape=(), dtype=float32)
tf.Tensor(0.37256306, shape=(), dtype=float32)


Uso de de una CNN de una capa 

In [3]:
salida_conv = tf.nn.conv2d(X_test,filtros,[1,2,2,1],"SAME")
salida_conv += b_conv
salidaPlana = tf.reshape(salida_conv,[-1,1568])

operacion_matricial = tf.matmul(salidaPlana,W)+b

probs = tf.nn.softmax(operacion_matricial)

max_preds = tf.argmax(probs,1)
max_trues = tf.argmax(y_test,1)

numCorrectos = tf.equal(max_preds,max_trues)
acc = tf.reduce_mean(tf.cast(numCorrectos,tf.float32)) # Tengo que convertirlo porque el original es booleano
print(acc)

tf.Tensor(0.8936, shape=(), dtype=float32)
